In [481]:
import pandas as pd
from langdetect import detect
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [482]:
# pip install yake
# pip install langdetect

In [483]:
fox = pd.read_csv('traderfox/traderfox.csv')
# fox['lang'] = fox['description'].apply(lambda x: detect(str(x)))
print(fox.columns)
len(fox)

Index(['name', 'description', '2021_Dividende', '2020_Dividende',
       '2019_Dividende', '2018_Dividende', '2017_Dividende', '2016_Dividende',
       '2015_Dividende', '2014_Dividende', '2013_Dividende', '2021_EBIT',
       '2020_EBIT', '2019_EBIT', '2018_EBIT', '2017_EBIT', '2016_EBIT',
       '2015_EBIT', '2014_EBIT', '2013_EBIT', '2021_EPS', '2020_EPS',
       '2019_EPS', '2018_EPS', '2017_EPS', '2016_EPS', '2015_EPS', '2014_EPS',
       '2013_EPS', '2021_Gewinn', '2020_Gewinn', '2019_Gewinn', '2018_Gewinn',
       '2017_Gewinn', '2016_Gewinn', '2015_Gewinn', '2014_Gewinn',
       '2013_Gewinn', '2021_KGV', '2020_KGV', '2019_KGV', '2018_KGV',
       '2017_KGV', '2016_KGV', '2015_KGV', '2014_KGV', '2013_KGV', '2021_KUV',
       '2020_KUV', '2019_KUV', '2018_KUV', '2017_KUV', '2016_KUV', '2015_KUV',
       '2014_KUV', '2013_KUV', '2021_Umsatz', '2020_Umsatz', '2019_Umsatz',
       '2018_Umsatz', '2017_Umsatz', '2016_Umsatz', '2015_Umsatz',
       '2014_Umsatz', '2013_Umsatz', '2022_

5419

In [484]:
tickers = pd.read_csv('traderfox/traderfox_ticker.csv')
tickers.columns = ['name2', 'ticker']
print(tickers.columns)
len(tickers)

Index(['name2', 'ticker'], dtype='object')


5419

In [485]:
sectors = pd.read_csv('stockslist.csv')
print(sectors.columns)
len(sectors)

Index(['Symbol', 'Company Name', 'Industry', 'Market Cap'], dtype='object')


6040

In [486]:
data = pd.concat([tickers[['ticker']], fox], axis=1)

In [487]:
fox_sectors = pd.read_csv('traderfox/traderfox_sectors.csv')
fox_sectors = pd.concat([fox_sectors, pd.read_csv('traderfox/traderfox_sectors_2.csv')], axis=0)
fox_sectors.columns = ['name3', 'sector']
print(fox_sectors.columns)
len(fox_sectors)

Index(['name3', 'sector'], dtype='object')


4870

In [488]:
fox_sectors = fox_sectors.drop_duplicates(subset=['name3'])
print(fox_sectors.columns)
len(fox_sectors)

Index(['name3', 'sector'], dtype='object')


4650

In [489]:
# 0 - 396           DAX
# 397 - 4600        NASDAQ & NYSE
# 4601 - 5194       EURO STOXX
# 5195 - end        NIKKEI

In [490]:
merged = pd.merge(data[397:4601], sectors, how='left', left_on='ticker', right_on='Symbol')
merged = pd.concat([data[:397], merged], axis=0)
merged = pd.merge(merged, fox_sectors, how='left', left_on='name', right_on='name3')

first_cols = ['ticker', 'name', 'description'] + list(sectors.columns) + list(fox_sectors.columns)
cols = first_cols + [col for col in merged.columns if col not in first_cols]
merged = merged[cols]

In [491]:
merged

,ticker,name,description,Symbol,Company Name,Industry,Market Cap,name3,sector,2021_Dividende,2020_Dividende,2019_Dividende,2018_Dividende,2017_Dividende,2016_Dividende,2015_Dividende,2014_Dividende,2013_Dividende,2021_EBIT,2020_EBIT,2019_EBIT,2018_EBIT,2017_EBIT,2016_EBIT,2015_EBIT,2014_EBIT,2013_EBIT,2021_EPS,2020_EPS,2019_EPS,2018_EPS,2017_EPS,2016_EPS,2015_EPS,2014_EPS,2013_EPS,2021_Gewinn,2020_Gewinn,2019_Gewinn,2018_Gewinn,2017_Gewinn,2016_Gewinn,2015_Gewinn,2014_Gewinn,2013_Gewinn,2021_KGV,2020_KGV,2019_KGV,2018_KGV,2017_KGV,2016_KGV,2015_KGV,2014_KGV,2013_KGV,2021_KUV,2020_KUV,2019_KUV,2018_KUV,2017_KUV,2016_KUV,2015_KUV,2014_KUV,2013_KUV,2021_Umsatz,2020_Umsatz,2019_Umsatz,2018_Umsatz,2017_Umsatz,2016_Umsatz,2015_Umsatz,2014_Umsatz,2013_Umsatz,2022_Dividende,2022_EBIT,2022_EPS,2022_Gewinn,2022_KGV,2022_KUV,2022_Umsatz
0,1U1,11-ag,Die 1&1 AG ist ein börsennotierter Telekommuni...,NaN,NaN,NaN,NaN,11-ag,Telekommunikation / Telekomdienstleister,0.0039,0.0039,0.0039,0.0039,0.1241,0.1396,0.1358,0.1319,0.1241,5.351200e+08,3.125900e+08,5.224400e+08,5.625800e+08,4.312900e+08,5.556000e+07,65800000.0,72610000.0,167670000.0,2.10,1.25,2.12,2.30,1.91,0.48,0.85,1.03,3.24,3.700200e+08,2.195900e+08,3.736200e+08,4.060300e+08,2.589300e+08,26430000.0,45800000.0,50080000.0,155770000.0,6.14,10.35,6.08,5.61,6.75,26.85,15.16,12.51,3.98,0.58,0.60,0.62,0.62,0.81,0.99,1.12,2.37,2.13,3.910000e+09,3.790000e+09,3.670000e+09,3.660000e+09,2.810000e+09,7.100100e+08,6.295500e+08,2.896500e+08,2.904700e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TGT,11-88-0-solutions-ag,Die 11 88 0 Solutions AG (ehemals telegate AG)...,NaN,NaN,NaN,NaN,11-88-0-solutions-ag,Telekommunikation / Telekomdienstleister,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3718,-6.400000e+05,-2.950000e+06,-2.360000e+06,-3.010000e+06,-9.870000e+06,-1.450000e+07,-10410000.0,-9110000.0,-2790000.0,0.04,-0.11,-0.16,-0.16,-0.49,-0.76,-0.47,-0.35,-0.07,1.040000e+06,-2.310000e+06,-3.180000e+06,-3.110000e+06,-9.630000e+06,-14670000.0,-9190000.0,-6770000.0,-1380000.0,25.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.47,0.52,0.47,0.48,0.50,0.46,0.38,0.33,0.28,5.653000e+07,5.080000e+07,4.767000e+07,4.292000e+07,4.127000e+07,4.471000e+07,5.354000e+07,6.226000e+07,7.233000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2GB,2g-energy-ag,2G Energy AG gehört zu den etablierten Unterne...,NaN,NaN,NaN,NaN,2g-energy-ag,-,0.0053,0.0048,0.0048,0.0048,0.0044,0.0042,0.0039,0.0039,0.0039,1.772000e+07,1.625000e+07,1.512000e+07,1.119000e+07,6.960000e+06,3.480000e+06,4490000.0,10940000.0,2800000.0,0.70,0.68,0.58,0.42,0.28,0.10,0.16,0.37,0.05,1.261000e+07,1.197000e+07,1.031000e+07,7.520000e+06,5.010000e+06,1800000.0,2780000.0,6490000.0,890000.0,33.57,34.91,40.55,55.66,83.69,231.37,149.37,64.48,472.00,1.59,1.69,1.77,1.99,2.21,2.40,2.74,2.24,3.32,2.663500e+08,2.467300e+08,2.364000e+08,2.097800e+08,1.894000e+08,1.743000e+08,1.528800e+08,1.866100e+08,1.261300e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UUU,3u-holding-ag,Die 3U HOLDING AG mit Sitz in Marburg wurde 19...,NaN,NaN,NaN,NaN,3u-holding-ag,Telekommunikation / Telekomdienstleister,0.0125,0.0125,0.0100,0.0075,0.0050,0.0025,NaN,NaN,NaN,6.290000e+06,5.260000e+06,4.640000e+06,1.830000e+06,1.900000e+06,1.000000e+05,-290000.0,-3330000.0,-5060000.0,0.08,0.10,0.12,0.06,0.03,0.02,-0.02,-0.09,-0.12,2.920000e+06,3.270000e+06,4.090000e+06,1.930000e+06,1.070000e+06,640000.0,-820000.0,-3240000.0,-4120000.0,48.07,42.00,32.44,68.79,133.00,199.50,NaN,NaN,NaN,2.52,2.31,2.65,2.76,2.82,3.02,2.74,2.74,3.49,5.594000e+07,6.105000e+07,5.145000e+07,4.797000e+07,4.689000e+07,4.374000e+07,4.824000e+07,4.924000e+07,3.971000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HRPK,7c-solarparken-ag,Die 7C Solarparken AG zählt zu den mittelgroße...,NaN,NaN,NaN,NaN,7c-solarparken-ag,Alternative Energieerzeugung / Solarenergie,0.0249,0.0249,0.0248,0.0247,0.0225,NaN,NaN,NaN,NaN,1.121000e+07,7.170000e+06,8.640000e+06,8.180000e+06,7.460000e+06,6.220000e+06,6310000.0,7820000.0,-3330000.0,0.14,0.08,0.14,0.12,0.14,0.11,0.16,0.28,-0.36,9.860000e+06,5.250000e+06,7.880000e

In [492]:
first_level = [str(i) for i in range(2013,2023)]
second_level = ['Dividende', 'EBIT', 'EPS', 'Gewinn', 'KGV', 'KUV', 'Umsatz']

# Combine the levels into a MultiIndex
multiindex = pd.MultiIndex.from_product([first_level, second_level])

In [ ]:
sel = sorted([col for col in merged.columns if col.startswith('2')])
facts = merged[[col for col in merged.columns if col not in sel]]
merged = merged[sel]
merged.columns = multiindex

In [494]:
first_level = ['company_info']
second_level = ['ticker', 'name', 'sector', 'description']

# Combine the levels into a MultiIndex
multiindex = pd.MultiIndex.from_product([first_level, second_level])

In [495]:
facts['name'] = facts['name'].apply(lambda x: ' '.join(x.split('-')))
facts['Company Name'] = facts['Company Name'].combine_first(facts['name'])
facts['Company Name'] = facts['Company Name'].apply(str.lower)
facts['Industry'] = facts['Industry'].combine_first(facts['sector'])

facts = facts[['ticker', 'Company Name', 'Industry', 'description']]
facts.columns = multiindex

/tmp/ipykernel_86190/1259925969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facts['name'] = facts['name'].apply(lambda x: ' '.join(x.split('-')))
/tmp/ipykernel_86190/1259925969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facts['Company Name'] = facts['Company Name'].combine_first(facts['name'])
/tmp/ipykernel_86190/1259925969.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [496]:
pd.concat([facts, merged], axis=1)[('company_info','sector')].value_counts(dropna=False)

Biotechnology                      494
Banks - Regional                   309
NaN                                291
Software - Application             148
Medical Devices                    104
                                  ... 
Konsumgüter / Bekleidung/Mode        1
Medizin / Medizindienstleister       1
Bauindustrie / Haus-/Gebäudebau      1
Konsumgüter / Luxusgüter             1
Telecom Services - Foreign           1
Name: (company_info, sector), Length: 219, dtype: int64

In [478]:
import yake

##### stem and lem inbefore!!!

kw_extractor = yake.KeywordExtractor()
text = """
Die 7C Solarparken AG zählt zu den mittelgroßen Betreibern von Solarparks in Deutschland. Alle laufenden Tätigkeiten erfolgen mit Eigenkompetenzen der Gesellschaft. 7C Solarparken verfügt über eine eigene O&M-Abteilung. Sämtliche administrative und unterstützende kaufmännische Tätigkeiten in den Bereichen Akquise, Finanzierung und Verbesserung der laufenden Betreibergesellschaften werden hausintern durchgeführt. Die 7C Solarparken AG ist im September 2014 aus der Übernahme der Colexon Energy AG durch die belgische 7C Solarparken NV hervorgegangen.
"""
language = "de"
max_ngram_size = 1
deduplication_threshold = 0.3
numOfKeywords = 10
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

('Solarparken', 0.04993372328294843)
('Deutschland', 0.0676124936337362)
('Betreibern', 0.08504893984356532)
('zählt', 0.2141628356917364)
('mittelgroßen', 0.2141628356917364)
('Eigenkompetenzen', 0.24404926327940085)
('Akquise', 0.3101897629925119)
('Finanzierung', 0.3101897629925119)
('Übernahme', 0.3968605949782225)
('Colexon', 0.3968605949782225)
